<a href="https://colab.research.google.com/github/RMoulla/legrand/blob/main/Copie_de_TP_RAG_Correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TP : Retrieval Augmented Generation (RAG)**

Dans ce TP, nous allons construire un **Retrieval Augmented Generation (RAG)**. L'objectif est de fournir une méthode complète pour traiter un fichier PDF, en extraire des informations pertinentes, puis les utiliser pour répondre à une requête utilisateur de manière augmentée. Ce TP est divisé en trois phases distinctes :

1. **Extraction et segmentation du texte** : Vous allez apprendre à extraire du texte à partir d'un fichier PDF et à le segmenter en sections de taille définie.
2. **Calcul des embeddings et recherche des segments similaires** : Nous utiliserons des techniques d'embeddings pour trouver les sections du texte les plus pertinentes par rapport à une requête donnée.
3. **Génération de réponses avec un modèle de type ChatGPT** : En utilisant le contexte extrait et le modèle GPT-4, nous allons générer une réponse contextualisée à la question posée par l'utilisateur.

Par ailleurs, nous allons utiliser des outils comme `pdfplumber`, pour parser les documents pdf, `sentence-transformers` pour les embeddings et l'API OpenAI pour générer des réponses augmentées par le contexte.

In [ ]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 63.5 MB/s eta 0:00:00


## **Étape 1 : Extraction et segmentation du texte**

Dans cette première étape, nous allons nous concentrer sur l'extraction du texte d'un fichier PDF et sa segmentation en blocs de taille définie. L'objectif est de rendre le texte extrait plus facile à traiter dans les phases suivantes.

### Détails :
1. **Extraction du texte** : À l'aide de `pdfplumber`, vous allez extraire tout le texte d'un fichier PDF donné. Cette étape vous permettra d'obtenir une version brute du contenu du document, qui servira de base pour les autres étapes.
   
2. **Segmentation du texte** : Une fois le texte extrait, vous le diviserez en segments de taille fixe (par exemple, 500 caractères par segment) à l'aide de la fonction `textwrap.wrap()` de Python. Cela permet de découper le texte en morceaux plus faciles à manipuler dans la phase suivante où nous calculerons les similarités.

Cette étape est cruciale, car elle permet de structurer le texte pour le rendre exploitable par la suite. La qualité de l'extraction et de la segmentation influencera directement les résultats des phases suivantes.

In [ ]:
import pdfplumber
import textwrap

# Chemin vers le fichier PDF
pdf_path = "MMD.pdf"

# Extraction du texte et des tables
full_text = ""
tables = []  # Liste pour stocker les tables extraites

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        # Extraction du texte
        page_text = page.extract_text()
        if page_text:
            full_text += page_text

        # Extraction des tables
        page_tables = page.extract_tables()
        if page_tables:
            tables.extend(page_tables)  # Ajouter les tables extraites à la liste

# Segmenter le texte en utilisant textwrap.wrap avec une longueur maximale de caractères

paragraphs = textwrap.wrap(full_text.strip(), width=500)

# Affichage des segments extraits
print(f"Nombre de segments de texte extraits : {len(paragraphs)}")
for i, para in enumerate(paragraphs[:5], 1):
    print(f"Segment {i}: {para}\n")

# Affichage des tables extraites
if tables:
    print(f"Nombre de tables extraites : {len(tables)}")
    print("Première table extraite :")
    for row in tables[0]:
        print(row)
else:
    print("Aucune table trouvée dans le PDF.")

Nombre de segments de texte extraits : 160
Segment 1: Texte mining par la pratique Redha Moulla Paris,14-16octobre2024 RedhaMoulla Paris,14-16octobre2024 1/151Plan de la formation Introduction au NLP Techniques statistiques pour le NLP Machine learning pour le NLP Deep learning pour le NLP Transformers et LLMs Alignement des LLMs RedhaMoulla Paris,14-16octobre2024 2/151Introduction au NLP RedhaMoulla Paris,14-16octobre2024 3/1511956 : conf´erence de Dartmouth L’histoire du NLP est intimement li´ee `a celle de l’intelligence artificielle. Il figure

Segment 2: mˆeme dans le programme de la conf´erence de Dartmouth, qui a fond´e l’IA. RedhaMoulla Paris,14-16octobre2024 4/151´ Evolution du NLP L’histoire du NLP peut ˆetre divis´ee en plusieurs phases cl´es, allant des approches symboliques aux r´evolutions r´ecentes avec les mod`eles de langage g´eants (LLMs). Ann´ees 1950 - 1980 : Approches symboliques Ann´ees 1990 - 2010 : Approches statistiques 2010 - 2018 : Deep Learning et r´eseaux d

## **Étape 2 : Calcul des Embeddings et recherche des segments similaires**

Dans cette deuxième étape, nous allons calculer des **embeddings** pour chaque segment de texte extrait et effectuer une recherche pour identifier les segments les plus similaires à une requête utilisateur donnée.

### Détails :
1. **Calcul des embeddings** : Nous allons utiliser la bibliothèque `sentence-transformers` pour générer des vecteurs d’embeddings pour chaque segment de texte. Ces vecteurs capturent les caractéristiques sémantiques des segments, permettant ainsi de comparer leur pertinence par rapport à une requête donnée.

2. **Recherche des segments les plus similaires** : Une fois les embeddings calculés, nous allons utiliser la **similarité cosinus** pour mesurer la proximité entre l’embedding de la requête utilisateur et les embeddings des segments de texte. Les segments les plus proches seront sélectionnés pour la prochaine étape.

3. **Sélection des top-N segments** : Nous sélectionnerons les `n` segments les plus similaires à la requête, qui serviront de contexte pour générer une réponse augmentée dans la phase suivante.

Cette étape est essentielle pour filtrer le contenu extrait en fonction de sa pertinence par rapport à la requête utilisateur. Elle permet de s'assurer que seuls les segments les plus utiles sont utilisés pour générer une réponse cohérente et contextuelle.

In [ ]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 4.1 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer, util

# Charger un modèle pré-entraîné de sentence-transformers
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavio

In [ ]:
# Calcul des embeddings pour chaque segmnt de texte (tableaux NumPy par défaut)
embeddings = model.encode(paragraphs)

In [ ]:

# Requête utilisateur pour laquelle nous cherchons les segments similaires
query = "Quel est le nombre de paramètres de GPT-3 ?"
query_embedding = model.encode(query)

# Calcul des similarités cosinus entre la requête et les segments
similarities = util.cos_sim(query_embedding, embeddings)

# Sélection des top-N documents les plus similaires
top_n = 10
top_results = similarities.topk(top_n)[1]  # Indices des top-N documents similaires

# Accéder aux indices des résultats top-N
top_n_paragraphs = [paragraphs[i] for i in top_results[0]]

# Affichage des top-N segments les plus similaires
for i, para in enumerate(top_n_paragraphs, 1):
    print(f"Segment {i} similaire : {para}\n")

Segment 1 similaire : 136/151Architecture de GPT-3 GPT-3 repose sur une architecture Transformer am´elior´ee, optimis´ee pour traiter efficacement de grandes quantit´es d’informations et g´en´erer des r´eponses coh´erentes et contextuellement pertinentes. Taille du mod`ele : Avec 175 milliards de param`etres, GPT-3 est l’un des mod`eles de langue les plus grands et les plus complexes `a ce jour. M´ecanisme d’attention : L’attention multi-tˆetes permet au mod`ele de pond´erer diff´eremment les parties d’un input,

Segment 2 similaire : validation crois´ee. Le nombre de variable tir´ees pour chaque noeud est g´en´eralement donn´e par √ p pour la classification et p pour la r´egression. Cet hyperparam`etre d´epend 3 cependant du probl`eme consid´er´e. Lorsque le nombre de vairable est ´elev´e alors que le nombre de variables r´eellement partinente est faible, la probabilit´e que les p variables s´electionn´ees pour chaque partitionnement incluent des variables pertinentes devient faible, 

## **Étape 3 : Génération de réponses avec ChatGPT**

Dans cette troisième, nous allons utiliser les segments de texte sélectionnés et une requête utilisateur pour générer une réponse contextuelle en utilisant l'API OpenAI, avec un modèle de type **ChatGPT**.

### Objectifs :
- Utiliser le modèle **GPT-4** pour générer une réponse basée sur les segments de texte les plus similaires à la requête.
- Envoyer les segments sélectionnés et la requête utilisateur sous forme de **messages** à l’API ChatGPT.
- Obtenir une réponse contextuelle, augmentée par les informations pertinentes extraites du texte.

### Détails :
1. **Création du contexte** : Nous allons concaténer les segments sélectionnés lors de l’étape précédente afin de créer un contexte cohérent à transmettre au modèle GPT-4. Ce contexte servira à fournir un maximum d’informations pertinentes pour générer une réponse précise.

2. **Appel à l’API ChatGPT** : En utilisant le contexte et la requête de l’utilisateur, nous formulerons un prompt que nous enverrons à l’API ChatGPT. Le prompt sera structuré sous forme de messages (avec les rôles "system" et "user") pour que le modèle comprenne le contexte de la conversation.

3. **Génération et récupération de la réponse** : Le modèle GPT-4 générera une réponse basée sur le contexte fourni. La réponse sera ensuite récupérée et affichée comme résultat final.

Cette étape finalise le processus de **RAG** en combinant la recherche d’informations pertinentes dans un corpus de texte et la génération de réponses intelligentes basées sur ces informations.

In [ ]:
!pip install openai==0.28

In [ ]:
import openai


openai.api_key = " "

# Concaténer les top-N segments en un seul contexte
context = "\n".join(top_n_paragraphs)
prompt = f"Contexte :\n{context}\n\nQuestion : {query}\nRéponse :"

# Appeler l'API d'OpenAI
response = openai.ChatCompletion.create(
    model="gpt-4o",  # Utilisation du modèle gpt-4o
    messages=[
        {"role": "system", "content": "Tu es un assistant specialisé dans la recherche d'information à partir de documents fournis. Tes réponses doivent absolument provenir du contexte fourni."},
        {"role": "user", "content": prompt}
    ],
    temperature=0
)

# Extraire et afficher la réponse générée
generated_response = response['choices'][0]['message']['content'].strip()
print(f"Réponse générée : {generated_response}")

Réponse générée : GPT-3 possède 175 milliards de paramètres.
